In [55]:
import tweepy
import pandas as pd

from code import utils, pipelines

In [54]:
credentials = utils.load_credentials('private/tweepy.json')

In [5]:
auth, api = utils.setup_tweepy(credentials)

In [6]:
f = open("data/media.txt")

In [7]:
lines = f.readlines()

In [8]:
def clean_line(line):
    line = line.split('\t')
    line[-1] = line[-1][:-1]
    
    return line

In [9]:
data = pd.DataFrame(columns=clean_line(lines[0]))

In [10]:
for line in lines[1:]:
    data = data.append(pd.Series(clean_line(line), index = data.columns), ignore_index=True)

In [11]:
data['screen_name'] = data['Twitter'].apply(lambda x: x[1:])
data.head()

,State,Chamber of Congress,Name,Party,Twitter,Instagram,Facebook Page,screen_name
0,Alabama,Senator,Richard Shelby,R,@SenShelby,@senatorshelby,x,SenShelby
1,Alabama,Senator,Doug Jones,D,@DougJones,@dougjonesbama,x,DougJones
2,Alabama 1st District,Representative,"Byrne, Bradley",R,@RepByrne,@repbyrne,x,RepByrne
3,Alabama 2nd District,U.S. Representative,"Roby, Martha",R,@RepMarthaRoby,@martharoby,x,RepMarthaRoby
4,Alabama 3rd District,U.S. Representative,"Rogers, Mike",R,@RepMikeRogersAL,@repmikerogersal,x,RepMikeRogersAL


In [12]:
data.drop_duplicates(subset=['Twitter'], inplace=True)

In [13]:
grouped_data = data.groupby('Party')

rep_media = grouped_data.get_group('R')
dem_media = grouped_data.get_group('D')

In [14]:
print(len(rep_media), len(dem_media))

251 282


In [15]:
rep_screen_names = rep_media['screen_name']
dem_screen_names = dem_media['screen_name']

In [31]:
# This takes some time
republican_likes, failed = pipelines.like_pipeline(api, rep_screen_names)

100%|██████████| 251/251 [8:23:41<00:00, 120.40s/it]   


In [36]:
import pickle

with open('republican_likes.pkl', 'wb') as rl:
    pickle.dump(republican_likes, rl)

In [51]:
len(republican_likes)

250

In [39]:
to_do = ['RepFredKeller', 'SenatorRomney']

In [40]:
republican_likes_to_add, failed = pipelines.like_pipeline(api, to_do)

100%|██████████| 2/2 [00:12<00:00,  6.48s/it]


In [46]:
len(republican_likes)

248

In [47]:
print(len(republican_likes))
republican_likes.extend(republican_likes_to_add)
print(len(republican_likes))

248
250


In [48]:
with open('republican_likes.pkl', 'wb') as rl:
    pickle.dump(republican_likes, rl)

In [49]:
with open('republican_likes.pkl', 'rb') as rl:
    tx = pickle.load(rl)

In [52]:
len(tx)

250

In [ ]:
failed = pipelines.tweet_pipeline(api, rep_screen_names, 'data/rep/')

In [ ]:
to_do = [fail[0] for fail in failed]

In [ ]:
republican_tweets_to_add = pipelines.tweet_pipeline(api, rep_screen_names, 'data/rep/')